In [1]:
#importo las librerias necesarias

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures

from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error

In [3]:
#cargo el dataset de training y el de test

df_training = pd.read_csv('./../data/training_one_hot_no_cities.csv')
df_test = pd.read_csv('./../data/diamonds_test_one_hot_no_cities.csv')
df_training

,price,carat,depth,table,x,y,z,clarity_I1,clarity_IF,clarity_SI1,...,color_F,color_G,color_H,color_I,color_J,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,3513,1.02,61.6,58.0,6.40,6.35,3.93,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1792,0.77,62.3,58.0,5.86,5.80,3.63,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,7553,1.51,59.6,60.0,7.58,7.48,4.49,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1176,0.57,60.2,62.0,5.40,5.33,3.23,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,1019,0.57,59.5,57.0,5.46,5.42,3.24,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
40451,6096,1.44,63.2,55.0,7.21,7.18,4.54,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
40452,3459,1.02,61.2,56.0,6.47,6.50,3.97,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
40453,4428,1.28,61.6,57.0,6.93,6.96,4.28,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
# Features + target

X = df_training[['carat',
                    'depth',
                    'table',
                    'x',
                    'y',
                    'z',
                    'cut/weight_Fair',
                    'cut/weight_Good',
                    'cut/weight_Ideal',
                    'cut/weight_Premium',
                    'cut/weight_Very Good',
                    'color/weight_D',
                    'color/weight_E',
                    'color/weight_F',
                    'color/weight_G',
                    'color/weight_H',
                    'color/weight_I',
                    'color/weight_J',
                    'clarity/weight_I1',
                    'clarity/weight_IF',
                    'clarity/weight_SI1',
                    'clarity/weight_SI2',
                    'clarity/weight_VS1',
                    'clarity/weight_VS2',
                    'clarity/weight_VVS1',
                    'clarity/weight_VVS2']]
y = pd.get_dummies(df_training[['price']], drop_first=True)
print(X.shape,y.shape)

KeyError: "['cut/weight_Fair', 'cut/weight_Good', 'cut/weight_Ideal', 'cut/weight_Premium', 'cut/weight_Very Good', 'color/weight_D', 'color/weight_E', 'color/weight_F', 'color/weight_G', 'color/weight_H', 'color/weight_I', 'color/weight_J', 'clarity/weight_I1', 'clarity/weight_IF', 'clarity/weight_SI1', 'clarity/weight_SI2', 'clarity/weight_VS1', 'clarity/weight_VS2', 'clarity/weight_VVS1', 'clarity/weight_VVS2'] not in index"

In [28]:
# Train + test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
# Scaling - StandardScaler
scaler = RobustScaler()
scaled_data = scaler.fit_transform(X_train)
scaled_data_test = scaler.transform(X_test)
scaled_data_test

array([[ 0.484375  ,  0.66666667,  0.33333333, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.484375  ,  0.73333333,  0.        , ...,  0.        ,
         0.99009901,  0.        ],
       [-0.625     , -0.93333333,  0.66666667, ...,  3.33333333,
         0.        ,  0.        ],
       ...,
       [-0.296875  ,  1.13333333, -0.33333333, ...,  1.96078431,
         0.        ,  0.        ],
       [ 1.296875  ,  0.06666667,  0.        , ...,  0.        ,
         0.        ,  0.65359477],
       [-0.546875  , -0.13333333, -0.33333333, ...,  0.        ,
         0.        ,  0.        ]])

In [30]:
regressor =  XGBRegressor()

In [31]:
hyperparameters = regressor.get_params()
model = regressor.fit(X_train, y_train.values.ravel())

In [32]:
y_pred = model.predict(X_test)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
#print('Ground truth target:', y_test, '\n')
print('Predicted target:', y_pred, '\n')

Model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...) 

Model hyperparameters: {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'early_stopping_rounds': None, 'enable_categorical': False

In [33]:
y_pred.shape

(8091,)

In [34]:
type(y_pred)

numpy.ndarray

In [35]:
r2_ = model.score(X_test, y_test)
r2_

0.9803647953480724

In [36]:
y_test = model.predict(df_test)

In [37]:
df_pred = pd.DataFrame(data=y_test)
df_pred

,0
0,2924.368896
1,5902.099121
2,9701.602539
3,3845.878174
4,1713.383057
...,...
13480,1624.106567
13481,2432.855957
13482,3012.078857
13483,2242.494141


In [16]:
df_pred = df_pred.reset_index()
df_pred

,index,0
0,0,2924.368896
1,1,5902.099121
2,2,9701.602539
3,3,3845.878174
4,4,1713.383057
...,...,...
13480,13480,1624.106567
13481,13481,2432.855957
13482,13482,3012.078857
13483,13483,2242.494141


In [17]:
df_rename = df_pred.rename(columns={"index": "id", "o": "price"})
df_rename

,id,0
0,0,2924.368896
1,1,5902.099121
2,2,9701.602539
3,3,3845.878174
4,4,1713.383057
...,...,...
13480,13480,1624.106567
13481,13481,2432.855957
13482,13482,3012.078857
13483,13483,2242.494141


In [18]:
final_prediction = df_rename.rename(columns={df_rename.columns[1]: "price" })
final_prediction

,id,price
0,0,2924.368896
1,1,5902.099121
2,2,9701.602539
3,3,3845.878174
4,4,1713.383057
...,...,...
13480,13480,1624.106567
13481,13481,2432.855957
13482,13482,3012.078857
13483,13483,2242.494141


In [22]:
export = final_prediction.to_csv('./../data/one_hot_robust_xgb_no_cities_operations.csv', index=False)

In [24]:
pd.read_csv('./../data/one_hot_robust_xgb_no_cities_operations.csv').head()

,id,price
0,0,2924.3690
1,1,5902.0990
2,2,9701.6030
3,3,3845.8782
4,4,1713.3830
